In [1]:
import pandas as pd
import sqlite3 as lite
from NNClassifier import TrainedClassifier
import os.path
import numpy as np

Load the trained classifier:

In [2]:
training_data = 'NNClassifier/tmp/therapeutic/training_data.npz'
trained_model = 'NNClassifier/tmp/therapeutic'

In [3]:
tc = TrainedClassifier(training_data,trained_model)

------------------------------
loading the translator
------------------------------
------------------------------
loading the args
------------------------------
------------------------------
initialization
------------------------------
------------------------------
loading the model
------------------------------


Next, lets' load our database of data scraped from google (which we plan to use to classify drugs).

In [4]:
#local database:
database = 'googDrugData.db'

In [5]:
with lite.connect(database) as con:
    df_goog = pd.read_sql('select Generic_Name, Drug_Class_Data_clean from goog_drug_class_data;',con)
#Do a bit of cleaning:
df_goog['Generic_Name'] = df_goog['Generic_Name'].apply(lambda x: x.strip())
df_goog

,Generic_Name,Drug_Class_Data_clean
0,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
1,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
2,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
3,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
4,SULFACETAMIDE SODIUM,\n\n sulfa antibiotic / corticosteroid combin...
5,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotic / cort...
6,SULFACETAMIDE SODIUM,\n\n sulfa antibiotics\n \n\n
7,SULFACETAMIDE SODIUM,\n\n Other\n \n\n sulfa antibiotics\n \...
8,SULFACETAMIDE SODIUM,\n\n Sulfonamide Antibacterial [EPC]\n \n
9,SULFACETAMIDE SODIUM,\n\n Sulfonamides [Chemical/Ingredient]\n \n


Next, let's load the database of classified drugs, and check how accurate our classifier is:

In [6]:
if os.path.isfile('drug_uses.csv'):
    df_uses = pd.read_csv('drug_uses.csv',index_col=0)
else:
    df_uses = pd.read_csv('https://query.data.world/s/81zz5jv9bpf4n70eoaef1lvjw',index_col=0)
df_uses

,drugname_brand,drugname_generic,anatomical,therapeutic,pharmacologic,chemical,substance,name
0,8-MOP,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,PSORALENS FOR TOPICAL USE,METHOXSALEN,METHOXSALEN
1,8-MOP,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE,PSORALENS FOR SYSTEMIC USE,METHOXSALEN,METHOXSALEN
2,OXSORALEN,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,PSORALENS FOR TOPICAL USE,METHOXSALEN,METHOXSALEN
3,OXSORALEN,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE,PSORALENS FOR SYSTEMIC USE,METHOXSALEN,METHOXSALEN
4,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE
5,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE CAVOXIL
6,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE LAUROXIL
7,ABILIFY,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE HYDRATE
8,ABILIFY DISCMELT,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE
9,ABILIFY DISCMELT,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,OTHER ANTIPSYCHOTICS,ARIPIPRAZOLE,ARIPIPRAZOLE CAVOXIL


Correlate the two databases:

In [7]:
df = pd.merge(df_uses[['drugname_generic','anatomical','therapeutic','pharmacologic']]
                      ,df_goog,
              left_on='drugname_generic',
              right_on='Generic_Name',
              how='inner').drop('Generic_Name',1).drop_duplicates(keep='first').reset_index(drop=True)
df

,drugname_generic,anatomical,therapeutic,pharmacologic,Drug_Class_Data_clean
0,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralens\n \n\n
1,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Other\n \n\n psoralens\n \n\n\n\n\...
2,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,"\n\n Antipsoriatic\n \n ,\n \n Hypopigmentat..."
3,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoabsorption [moa]\n \n\n
4,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n photoactivated Radical Generator [epc]\n...
5,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n photosensitizing Activity [pe]\n \n\n
6,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralen [epc]\n \n\n
7,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n psoralens [chemical/ingredient]\n \n\n
8,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoabsorption [MoA]\n \n
9,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,\n\n Photoactivated Radical Generator [EPC]\n \n


In [8]:
df_testing = df[['drugname_generic','anatomical',
                 'therapeutic','pharmacologic']].drop_duplicates(keep='first').reset_index(drop=True)
df_testing

,drugname_generic,anatomical,therapeutic,pharmacologic
0,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE
1,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE
2,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS
3,ISOTRETINOIN,DERMATOLOGICALS,ANTI-ACNE PREPARATIONS,ANTI-ACNE PREPARATIONS FOR TOPICAL USE
4,ISOTRETINOIN,DERMATOLOGICALS,ANTI-ACNE PREPARATIONS,ANTI-ACNE PREPARATIONS FOR SYSTEMIC USE
5,ACARBOSE,ALIMENTARY TRACT AND METABOLISM,DRUGS USED IN DIABETES,"BLOOD GLUCOSE LOWERING DRUGS, EXCL. INSULINS"
6,ZAFIRLUKAST,RESPIRATORY SYSTEM,DRUGS FOR OBSTRUCTIVE AIRWAY DISEASES,OTHER SYSTEMIC DRUGS FOR OBSTRUCTIVE AIRWAY DI...
7,ACETAZOLAMIDE,SENSORY ORGANS,OPHTHALMOLOGICALS,ANTIGLAUCOMA PREPARATIONS AND MIOTICS
8,ACETIC ACID,GENITO URINARY SYSTEM AND SEX HORMONES,GYNECOLOGICAL ANTIINFECTIVES AND ANTISEPTICS,"ANTIINFECTIVES AND ANTISEPTICS, EXCL. COMBINAT..."
9,ACETIC ACID,SENSORY ORGANS,OTOLOGICALS,ANTIINFECTIVES


Run the classifier:

In [9]:
df_testing['Therapeutic_Classifier_Guess'] = ''
for i,drugname in enumerate(df_testing['drugname_generic']):
    descriptors = df[df['drugname_generic'] == drugname]['Drug_Class_Data_clean']
    #print(drugname,tc.top_classification_aggregate(descriptors))
    df_testing['Therapeutic_Classifier_Guess'].iloc[i] = tc.top_classification_aggregate(descriptors)
df_testing

,drugname_generic,anatomical,therapeutic,pharmacologic,Therapeutic_Classifier_Guess
0,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR TOPICAL USE,ANTIPSORIATICS
1,METHOXSALEN,DERMATOLOGICALS,ANTIPSORIATICS,ANTIPSORIATICS FOR SYSTEMIC USE,ANTIPSORIATICS
2,ARIPIPRAZOLE,NERVOUS SYSTEM,PSYCHOLEPTICS,ANTIPSYCHOTICS,PSYCHOLEPTICS
3,ISOTRETINOIN,DERMATOLOGICALS,ANTI-ACNE PREPARATIONS,ANTI-ACNE PREPARATIONS FOR TOPICAL USE,ANTI-ACNE PREPARATIONS
4,ISOTRETINOIN,DERMATOLOGICALS,ANTI-ACNE PREPARATIONS,ANTI-ACNE PREPARATIONS FOR SYSTEMIC USE,ANTI-ACNE PREPARATIONS
5,ACARBOSE,ALIMENTARY TRACT AND METABOLISM,DRUGS USED IN DIABETES,"BLOOD GLUCOSE LOWERING DRUGS, EXCL. INSULINS",DRUGS USED IN DIABETES
6,ZAFIRLUKAST,RESPIRATORY SYSTEM,DRUGS FOR OBSTRUCTIVE AIRWAY DISEASES,OTHER SYSTEMIC DRUGS FOR OBSTRUCTIVE AIRWAY DI...,DRUGS FOR OBSTRUCTIVE AIRWAY DISEASES
7,ACETAZOLAMIDE,SENSORY ORGANS,OPHTHALMOLOGICALS,ANTIGLAUCOMA PREPARATIONS AND MIOTICS,ANTIEPILEPTICS
8,ACETIC ACID,GENITO URINARY SYSTEM AND SEX HORMONES,GYNECOLOGICAL ANTIINFECTIVES AND ANTISEPTICS,"ANTIINFECTIVES AND ANTISEPTICS, EXCL. COMBINAT...",OTOLOGICALS
9,ACETIC ACID,SENSORY ORGANS,OTOLOGICALS,ANTIINFECTIVES,OTOLOGICALS


Check the accuracy, how often do we get one of the possible correct classifications?

In [11]:
correct = dict()
for drug in set(df_testing['drugname_generic']):
    df_tmp = df_testing[df_testing['drugname_generic']==drug]
    correct[drug] = (df_tmp['therapeutic'] == df_tmp['Therapeutic_Classifier_Guess']).sum()>0

In [14]:
total_correct = 0
for drug,cor in correct.items():
    total_correct += cor
accuracy = total_correct/len(correct)
print('Accuracy: %f'%accuracy)

Accuracy: 0.704100


In [10]:
mask = (df_testing['therapeutic'] == df_testing['Therapeutic_Classifier_Guess'])
accuracy = mask.sum()/mask.shape
print('Accuracy: %f'%accuracy)

Accuracy: 0.535286


In [17]:
num_entries = dict()
for drug in df_testing[mask == False]['drugname_generic']:
    if drug not in num_entries:
        num_entries[drug] = (df_testing['drugname_generic'] == drug).sum()
total_entries = 0
for drug,num in num_entries.items():
    total_entries += num
total_entries/len(num_entries)

1.8043478260869565